## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-14-13-02-25 +0000,bbc,US group pulls out of deal to buy Daily Telegraph,https://www.bbc.com/news/articles/c993drmj1dyo...
1,2025-11-14-13-00-50 +0000,bbc,What taxes could go up in Budget if income tax...,https://www.bbc.com/news/articles/cj4w44w42j5o...
2,2025-11-14-13-00-28 +0000,bbc,Doctor strike inflicting pain and misery on pa...,https://www.bbc.com/news/articles/cm2lg8pgjymo...
3,2025-11-14-13-00-10 +0000,nyt,"In a Brutal Mississippi Jail, Inmates Say They...",https://www.nytimes.com/2025/11/14/us/mississi...
4,2025-11-14-13-00-00 +0000,nypost,Inside the Merchant Marine-Coast Guard rivalry...,https://nypost.com/2025/11/14/sports/inside-th...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
451,trump,31
138,new,19
390,california,15
526,epstein,10
331,first,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
304,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...,98
284,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...,69
224,2025-11-13-21-21-08 +0000,nypost,BBC apologizes to Trump over selectively edite...,https://nypost.com/2025/11/13/us-news/bbc-apol...,66
139,2025-11-14-03-04-50 +0000,nypost,Va. prosecutor accused of ‘stymying the invest...,https://nypost.com/2025/11/13/us-news/va-prose...,65
314,2025-11-13-15-27-26 +0000,bbc,What we know about new Epstein emails that men...,https://www.bbc.com/news/articles/c3rj0d97ynvo...,65


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
304,98,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...
128,52,2025-11-14-04-38-11 +0000,nypost,JD Vance says surge of illegal immigrants drov...,https://nypost.com/2025/11/13/us-news/jd-vance...
218,51,2025-11-13-21-46-58 +0000,nypost,Stocks have worst day in a month amid AI conce...,https://nypost.com/2025/11/13/business/stocks-...
136,50,2025-11-14-03-37-55 +0000,nypost,NYC Mayor-elect Zohran Mamdani holds secretive...,https://nypost.com/2025/11/13/us-news/nyc-mayo...
284,38,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...
19,38,2025-11-14-12-09-00 +0000,wsj,Swiss Franc Hits Record High on Trade-Deal Hop...,https://www.wsj.com/finance/currencies/asian-c...
37,36,2025-11-14-11-24-34 +0000,nypost,Friends of NYC designer Martha Nolan joked abo...,https://nypost.com/2025/11/14/us-news/nyc-desi...
291,36,2025-11-13-17-01-23 +0000,nypost,Rep. Anna Paulina Luna launches probe into wil...,https://nypost.com/2025/11/13/us-news/rep-anna...
156,36,2025-11-14-01-18-01 +0000,nypost,Canada man hijacks city bus but miraculously m...,https://nypost.com/2025/11/13/world-news/canad...
165,32,2025-11-14-00-17-09 +0000,nyt,Government Shutdown Ends and Federal Workers R...,https://www.nytimes.com/2025/11/13/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
